# AKSSF Hydrography
## Construct Synthetic Stream Network and Catchments using TauDEM tools and Timm's 10m composite DEM clipped to Study Area
Stream Burning only necessary for Kodiak region.  The composite DEM for this region has some NoData gaps that must be
filled prior to running.

### Define Hydroprocessing Function
Create synthetic stream network and calculate stream reach and watersheds(catchments) using TauDEM tools and <a href = "https://downloads.esri.com/archydro/archydro/Setup/Pro/">ArcHydro tools</a> so these must me installed prior to running this code.

1. Clip input 10 meter composite DEM to study area and fill pits
    * Burn in AKHydro Carto/named streams from NHD to ensure that larger systems are identified.
        * Use archydro DEMreconditioning with 5 cell buffer, and 10-100 unit drops respectively.  Calling the tool is not straightforward, see documentation <a href = "https://community.esri.com/t5/water-resources-questions/calling-arc-hydro-tools-in-python-in-arcgis-pro/td-p/514773/page/3">here</a> and <a href = "https://community.esri.com/t5/water-resources-documents/arc-hydro-calling-arc-hydro-tools-in-python-pdf/ta-p/920821">pdf here</a>. 
2. Calculate D8 flow direction grid
    * This process takes the longest
3. Calculate D8 contributing area
4. Generate stream grid using Peuker Douglass Stream Definition
    * Accumulation threshold = 75
    * Minimum threshold = 20
    * Maximum threshold = 500
    * Number threshold values = 15
5. Process stream reach and watersheds
6. Export watershed grid to shapefile
    a. Dissolve watersheds on gridcode because TauDEM does not

### Inputs:
1. study_area: A polygon feature class with the field "Processing_Region" that has been coded to designate unique
watersheds to be processed iteratively.

2. TauLoc: path to TauDEM toolbox on local machine <a href = "https://hydrology.usu.edu/taudem/taudem5/"> TauDEM
</a> must be installed.

3. DEM:  path to 10 meter composite DEM covering study area.  DEM will be clipped to the study area polygon during
processing.

4. datafolder: path to Output folder where data will be saved.

5. Stream layer to burn into DEM.  Using AKHydro carto layer/named streams.


In [1]:
def hydroprocess(**kwargs):
    # Import packages
    import os
    import arcpy
    import datetime
    import time
    import archydro
    import demreconditioning
   
    """
    Description: Creates synthetic stream network and catchments from input DEM reconditioned with stream network
    :param Input_Number_of_Processes: 10
    :param rasclip: rasclip #Dem clipped to study area
    :param Check_for_edge_contamination:False #Dem clipped to watershed boundary so keep false
    :param Outputs: region_out # name of processing region
    :param _Name_catchments_shp: catchname
    :param Output_Stream_Reach_file: streamname
    :param streamburnnet: streamburnnet
    :return:
    """

    
    # Import TauDEM tools
    arcpy.ImportToolbox(TauLoc)

    # Use two thirds of cores on processes that can be split.
    arcpy.env.parallelProcessingFactor = "66%"

    # Parse key word argument inputs
    outfol = kwargs['outfol']
    rasclip = kwargs['rasclip']
    region = kwargs['region']
    Input_Number_of_Processes = kwargs['Input_Number_of_Processes']
    Check_for_edge_contamination = kwargs['Check_for_edge_contamination']
    streamburnnet = kwargs['streamburnnet']
    
    # To allow overwriting outputs change overwriteOutput option to True.
    arcpy.env.overwriteOutput = True
    arcpy.env.cellSize = 10
    arcpy.env.extent=rasclip
    arcpy.env.mask=rasclip
    arcpy.env.snapRaster=rasclip
    
    # Process: Convert stream network to raster and recondition DEM
    streamburn_rastername = region + "_streamburn_rast.tif"
    streamburn_rast = os.path.join (outfol, streamburn_rastername)
    line2rast_name = region + "streamburn_2rast.tif"
    line2rast = os.path.join (outfol, line2rast_name)
    streamburn_clipname = region + "_streamnetClip.shp"
    streamburn_clip = os.path.join (outfol, streamburn_clipname)
    if not arcpy.Exists(streamburn_rast):
        # Start timing function
        iteration_start = time.time()
        # Clip input stream burn feature to region of interest and add field for raster conversion
        arcpy.analysis.Clip(streamburnnet, roilayer, streamburn_clip)
        arcpy.management.AddField(in_table=streamburn_clip, field_name="RasterCat", field_type="LONG", field_precision=None, field_scale=None,
                                  field_length=None, field_alias="", field_is_nullable="NULLABLE", field_is_required="NON_REQUIRED",
                                  field_domain="")
        arcpy.management.CalculateField(in_table=streamburn_clip, field="RasterCat", expression=1, field_type="TEXT") 
        # Process: Polyline to Raster (Polyline to Raster)
        arcpy.conversion.PolylineToRaster(in_features=streamburn_clip, value_field="RasterCat", out_rasterdataset=line2rast,
                                          cell_assignment="MAXIMUM_LENGTH", priority_field="NONE", cellsize=10,
                                          build_rat="BUILD")
        # Process: Recondition DEM using DEM Recondition Agree Method (archydro tool)
        # arcpy.archydropro.DEMReconditioning(rasclip,line2rast, 3, 5, 10,streamburn_rast)# cannot figure out how to call archydropro tools?
        p_demrec = demreconditioning.DEMReconditioning()  
        p_demrec.bCallFromPYT = False # Don't know what this does but the esri example for importing archydro tools includes this line
        params = (rasclip, line2rast, 5, 10, 100,streamburn_rast)
        ret_demrec = p_demrec.execute(params, None)
        # End timing
        iteration_end = time.time()
        iteration_elapsed = int(iteration_end - iteration_start)
        iteration_success_time = datetime.datetime.now()
        # Report success
        print(f'Streams Burned at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
        print('----------')
    else:
        print("Stream Burn Raster already exists ")
        print('----------')
    streamburn_rast = arcpy.Raster(streamburn_rast)

    
    
    # Process: Pit Remove
    DEM_fillname = region + "_fel.tif"
    DEM_fill = os.path.join(outfol, DEM_fillname)

    if not arcpy.Exists(DEM_fill):
        # Start timing function
        iteration_start = time.time()
        arcpy.PitRemove(Input_Elevation_Grid=streamburn_rast, Fill_Considering_only_4_way_neighbors=False,
                         Input_Depression_Mask_Grid="", Input_Number_of_Processes=Input_Number_of_Processes,
                         Output_Pit_Removed_Elevation_Grid=DEM_fill)
        # End timing
        iteration_end = time.time()
        iteration_elapsed = int(iteration_end - iteration_start)
        iteration_success_time = datetime.datetime.now()
        # Report success
        print(f'Pits filled at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
        print('----------')
    else:
        print("Pit Filled Elevation Grid already exists ")
        print('----------')
    DEM_fill = arcpy.Raster(DEM_fill)

    # Process: D8 Flow Directions (D8 Flow Directions) ()
    _Name_p_tifname = region + "_p.tif"
    _Name_sd8_tifname = region + "_sd8.tif"
    _Name_p_tif = os.path.join(outfol, _Name_p_tifname)
    _Name_sd8_tif = os.path.join(outfol,_Name_sd8_tifname)
    if not arcpy.Exists(_Name_p_tif):
        # Start timing function
        iteration_start = time.time()
        arcpy.D8FlowDir(Input_Pit_Filled_Elevation_Grid=DEM_fill,
                         Input_Number_of_Processes=Input_Number_of_Processes,
                         Output_D8_Flow_Direction_Grid=_Name_p_tif, Output_D8_Slope_Grid=_Name_sd8_tif)
        # End timing
        iteration_end = time.time()
        iteration_elapsed = int(iteration_end - iteration_start)
        iteration_success_time = datetime.datetime.now()
        # Report success
        print(f'Flow Direction calculated at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
        print('----------')
    else:
        print("Flow Direction Grid already exists")
        print('----------')
    _Name_p_tif = arcpy.Raster(_Name_p_tif)
    _Name_sd8_tif = arcpy.Raster(_Name_sd8_tif)

    # Process: D8 Contributing Area (D8 Contributing Area) ()
    _Name_ad8_tifname = region + "_ad8.tif"
    _Name_ad8_tif = os.path.join(outfol, _Name_ad8_tifname)
    if not arcpy.Exists(_Name_ad8_tif):
        # Start timing function
        iteration_start = time.time()
        arcpy.D8ContributingArea(Input_D8_Flow_Direction_Grid=_Name_p_tif,
                                  Input_Outlets="", Input_Weight_Grid="",
                                  Check_for_edge_contamination=Check_for_edge_contamination,
                                  Input_Number_of_Processes=Input_Number_of_Processes,
                                  Output_D8_Contributing_Area_Grid=_Name_ad8_tif)
        # End timing
        iteration_end = time.time()
        iteration_elapsed = int(iteration_end - iteration_start)
        iteration_success_time = datetime.datetime.now()
        # Report success
        print(f'D8 Contributing area calculated at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
        print('----------')
    else:
        print ('D8 Contributing Area Grid Already Exists')
        print('----------')
    _Name_ad8_tif = arcpy.Raster(_Name_ad8_tif)

    # Process: Peuker Douglas Stream Definition (Peuker Douglas Stream Definition) ()
    _Name_ss_tifname = region + "_ss.tif"
    _Name_ssa_tifname = region + "_ssa.tif"
    _Name_src_tifname = region + "_src.tif"
    _Name_drp_txtname = region + "_drp.txt"
    _Name_ss_tif = os.path.join(outfol, _Name_ss_tifname)
    _Name_ssa_tif = os.path.join(outfol, _Name_ssa_tifname)
    _Name_src_tif = os.path.join(outfol, _Name_src_tifname)
    _Name_drp_txt = os.path.join(outfol, _Name_drp_txtname)

    if not arcpy.Exists(_Name_src_tif):
        # Start timing function
        iteration_start = time.time()
        arcpy.PeukerDouglasStreamDef(Input_Elevation_Grid=DEM_fill,
                                      Input_D8_Flow_Direction_Grid=_Name_p_tif,
                                      Weight_Center=0.4, Weight_Side=0.1, Weight_Diagonal=0.05,
                                      Accumulation_Threshold=75,
                                      Check_for_Edge_Contamination=Check_for_edge_contamination,
                                      Input_Outlets="", Input_Mask_Grid="",
                                      Input_D8_Contributing_Area_for_Drop_Analysis=_Name_ad8_tif,
                                      Input_Number_of_Processes=Input_Number_of_Processes,
                                      Output_Stream_Source_Grid=_Name_ss_tif,
                                      Output_Accumulated_Stream_Source_Grid=_Name_ssa_tif,
                                      Output_Stream_Raster_Grid=_Name_src_tif,
                                      Output_Drop_Analysis_Table=_Name_drp_txt,
                                      Use_the_range_below_to_automatically_select_threshold_by_drop_analysis=True,
                                      Minimum_Threshold_Value=20, Maximum_Threshold_Value=500,
                                      Number_of_Threshold_Values=15,
                                      Use_Logarithmic_spacing_for_threshold_values=True)
        # End timing
        iteration_end = time.time()
        iteration_elapsed = int(iteration_end - iteration_start)
        iteration_success_time = datetime.datetime.now()
        # Report success
        print(f'Stream Definition completed at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
        print('----------')
    else:
        print('Streams Already Defined')
        print('----------')
    _Name_ss_tif = arcpy.Raster(_Name_ss_tif)
    _Name_ssa_tif = arcpy.Raster(_Name_ssa_tif)
    _Name_src_tif = arcpy.Raster(_Name_src_tif)

    # Process: Stream Reach And Watershed (Stream Reach And Watershed) ()
    Output_Stream_Reach_file = region + '_strbrn_reach.shp'
    Output_Stream_Order_Gridname = region + "_ord.tif"
    Output_Network_Connectivity_Treename = region + "_tree.txt"
    Output_Network_Coordinatesname = region + "_coord.txt"
    Output_Watershed_Gridname = region + "_w.tif"
    Output_Stream_Order_Grid = os.path.join(outfol, Output_Stream_Order_Gridname)
    Output_Network_Connectivity_Tree =  os.path.join(outfol, Output_Network_Connectivity_Treename)
    Output_Network_Coordinates = os.path.join(outfol, Output_Network_Coordinatesname)
    Output_Watershed_Grid = os.path.join(outfol, Output_Watershed_Gridname)

    if not arcpy.Exists(Output_Stream_Order_Grid):
        # Start timing function
        iteration_start = time.time()
        arcpy.StreamReachAndWatershed(Input_Pit_Filled_Elevation_Grid=DEM_fill,
                                       Input_D8_Flow_Direction_Grid=_Name_p_tif,
                                       Input_D8_Drainage_Area=_Name_ad8_tif,
                                       Input_Stream_Raster_Grid=_Name_src_tif,
                                       Input_Outlets="", Delineate_Single_Watershed=False,
                                       Input_Number_of_Processes=Input_Number_of_Processes,
                                       Output_Stream_Order_Grid=Output_Stream_Order_Grid,
                                       Output_Network_Connectivity_Tree=Output_Network_Connectivity_Tree,
                                       Output_Network_Coordinates=Output_Network_Coordinates,
                                       Output_Stream_Reach_file=Output_Stream_Reach_file,
                                       Output_Watershed_Grid=Output_Watershed_Grid)
        # End timing
        iteration_end = time.time()
        iteration_elapsed = int(iteration_end - iteration_start)
        iteration_success_time = datetime.datetime.now()
        # Report success
        print(f'Stream Reach and Watershed completed at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
        print('----------')
    else:
        print ('Stream Reach and Watershed already identified')
        print('----------')
    Output_Stream_Order_Grid = arcpy.Raster(Output_Stream_Order_Grid)
    # Add Dissolve
    Output_Watershed_Grid = arcpy.Raster(Output_Watershed_Grid)

    # Process: Watershed Grid To Shapefile
    _Name_catchments_shpname = region + "_strbrn_watersheds.shp"
    _Name_catchments_shp = os.path.join(outfol, _Name_catchments_shpname)
    watersheds_name = region + "_strbrn_watersheds_diss.shp"
    watersheds_diss = os.path.join(outfol, watersheds_name)
              
    if not arcpy.Exists(_Name_catchments_shp):
        # Start timing function
        iteration_start = time.time()
        arcpy.WaterShedGridToShapefile(Input_Watershed_Grid=Output_Watershed_Grid,
                                       Output_Watershed_Shapefile=_Name_catchments_shp)
        # Dissolve Watersheds on gridcode
        arcpy.management.Dissolve(_Name_catchments_shp, watersheds_diss, 'gridcode')
        # Add processing region ID field
        arcpy.management.AddField(in_table=watersheds_diss, field_name="proc_reg", field_type="TEXT", field_precision=None, field_scale=None,
                                  field_length=None, field_alias="hydro processing region", field_is_nullable="NULLABLE", field_is_required="NON_REQUIRED",
                                  field_domain="")
        arcpy.management.CalculateField(in_table=streamburn_clip, field="proc_reg", expression=region, field_type="TEXT") 
        # End timing
        iteration_end = time.time()
        iteration_elapsed = int(iteration_end - iteration_start)
        iteration_success_time = datetime.datetime.now()
        # Report success
        print(f'Watershed Exported at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
        print('----------')
    else:
        print("Watershed Grid already exported to shapefile")
        print('----------')

# Set Inputs


In [1]:
import arcpy
import os

# path to TauDEM toolbox on local machine
TauLoc = r"C:\Program Files\TauDEM\TauDEM5Arc\TauDEM Tools.tbx"

# 10m Composite DEM Clipped to Study Area
#DEM = r"C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\data\\topography\\AKSSF_Elevation_Composite_10m.tif"

# DEM for Kodiak that has voids filled
DEM = r"C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\data\\topography\\AKSSF_Elevation_Composite_10m_VoidFill_Kodiak_Afognak.tif"

# Input feature class describing study area and processing regions that can be iterated over
study_area = r"C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\AKSSF_Hydrography.gdb\\AKKSSF_studyarea_HUC8"

# Folder to store outputs
datafolder = r"C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\data\\Outputs\\TauDEM\\V2_CartoStreamBurn"

# Input feature class representing streams to Burn into DEM
streamburnnet = r"D:\\Basedata\\AK_NHD_Carto.gdb\\NHD_named"

inputs = [TauLoc, DEM, study_area, datafolder, streamburnnet]
for input in inputs:
    if not arcpy.Exists(input):
        print('Please check that the path for {} is correct'.format(input))
        print('\t---------')
    else:
        print('Input {} set'.format(input))
        print('\t---------')





Input C:\Program Files\TauDEM\TauDEM5Arc\TauDEM Tools.tbx set
	---------
Input C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\data\\topography\\AKSSF_Elevation_Composite_10m.tif set
	---------
Input C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\AKSSF_Hydrography.gdb\\AKKSSF_studyarea_HUC8 set
	---------
Input C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\data\\Outputs\\TauDEM\\V2_CartoStreamBurn set
	---------
Input D:\\Basedata\\AK_NHD_Carto.gdb\\NHD_named set
	---------


# Begin Processing

In [6]:
import datetime
import time
arcpy.env.overwriteOutput = True
# Run only for Kodiak at this time
proc_list = ['KodiakAfognakIslands']

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

for region in proc_list:
    outfol = os.path.join(datafolder, region)
    if not arcpy.Exists(outfol):
        os.makedirs(outfol)
        print('{} output location created at {}'.format(region, outfol))
        print('\t----------')
    else:
        print('{} output location already exists at {}'.format(region, outfol))
        print('\t----------')

    arcpy.env.workspace = outfol
    # Create roi layer
    layer = region + "_layer"
    print(layer)
    print('\t----------')

    # Create expression
    reg = "'" + region + "'"
    print(reg)
    print('\t----------')

    exp = "Processing_Region = {}".format(reg)
    print(exp)
    print('\t----------')
    rasclip = os.path.join(outfol, region + "_10mCompDEMClip.tif")
    roilayer = arcpy.MakeFeatureLayer_management(study_area, layer, exp)
    if not arcpy.Exists(rasclip):
        print("Clipping DEM to {} region ".format(region))
        print("\t-----------")
        arcpy.management.Clip(in_raster = DEM,rectangle = "", out_raster = rasclip,
                              in_template_dataset = roilayer, nodata_value = -32768,
                              clipping_geometry = "ClippingGeometry",
                              maintain_clipping_extent = "NO_MAINTAIN_EXTENT")
    else:
        print("{} Processing Region already exists ".format(region))
        print("\t-----------")
    print(f'Begin Hydro Processing at {processStartdt.strftime("%Y-%m-%d %H:%M")}')
    # Keyword arguments for hydro processing
    hydroprocess_kwargs = {'outfol': outfol,
                           'rasclip':rasclip,
                           'region':region,
                           'Input_Number_of_Processes':10,
                           'Check_for_edge_contamination':False,
                           'streamburnnet':streamburnnet                           
                          }

    hydroprocess(**hydroprocess_kwargs)
# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()
# Report success

print(f'Hydronetwork(s) complete at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print('----------')


KodiakAfognakIslands output location already exists at C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\data\\Outputs\\TauDEM\\V2_CartoStreamBurn\KodiakAfognakIslands
	----------
KodiakAfognakIslands_layer
	----------
'KodiakAfognakIslands'
	----------
Processing_Region = 'KodiakAfognakIslands'
	----------
KodiakAfognakIslands Processing Region already exists 
	-----------
Begin Hydro Processing at 2021-05-14 11:02
VALIDATION_OK
Package archydro loaded from c:\users\dwmerrigan\appdata\local\programs\arcgis\pro\Resources\ArcToolbox\Scripts for user dwmerrigan dt=9.501s


  File "C:\Program Files\TauDEM\TauDEM5Arc\TauDEM Tools.tbx#ArcGISStabilityIndex.InitializeParameters.py", line 23
    mxd = arcpy.mapping.MapDocument("CURRENT")
                                             ^
TabError: inconsistent use of tabs and spaces in indentation
  File "C:\Program Files\TauDEM\TauDEM5Arc\TauDEM Tools.tbx#ParameterRegionTool.InitializeParameters.py", line 27
    self.params[2].enabled = 0
                             ^
TabError: inconsistent use of tabs and spaces in indentation
  File "C:\Program Files\TauDEM\TauDEM5Arc\TauDEM Tools.tbx#ArcGISStabilityIndex.InitializeParameters.py", line 23
    mxd = arcpy.mapping.MapDocument("CURRENT")
                                             ^
TabError: inconsistent use of tabs and spaces in indentation
  File "C:\Program Files\TauDEM\TauDEM5Arc\TauDEM Tools.tbx#ArcGISStabilityIndex.InitializeParameters.py", line 23
    mxd = arcpy.mapping.MapDocument("CURRENT")
                                             ^
TabError: inco

Streams Burned at 2021-05-14 11:13 (Elapsed time: 0:11:06)
----------
Pits filled at 2021-05-14 11:15 (Elapsed time: 0:02:17)
----------
Flow Direction calculated at 2021-05-14 13:08 (Elapsed time: 1:53:04)
----------
D8 Contributing area calculated at 2021-05-14 13:09 (Elapsed time: 0:01:08)
----------
Stream Definition completed at 2021-05-14 13:12 (Elapsed time: 0:02:11)
----------
Stream Reach and Watershed completed at 2021-05-14 13:14 (Elapsed time: 0:02:32)
----------


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000670: output Output Feature Class is same as input Input Features
Failed to execute (Dissolve).
